In [183]:
# Note that the initial license file from Colorado was too big to load into git hub.
# I ran earlier notebooks to analyze the dataset and determine the best way to reduce it.
# The original 1.2 million line professional license file was reduced to two files a) anesthesia and b) registereed nurse
# This notebook picks-up with a smaller license_anesthesia file that relates just to the non-physician anesthesia clinicians
# and a smaller license_regnurse file that relates to the underlying RN license file that supports practicing as a CRNA

# Rodgers Secondary Extract & Transform Workbook

In [310]:
# Initial Extract & Transform Dependencies
import pandas as pd
import time

# Secondardy PGAdmin Extract Dependencies
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Float 
from config import passcode

In [273]:
# Import the file of non-physician anesthesia clinicians
anesthesia=pd.read_csv('license_anesthesia.csv')
anesthesia.head()

,lastName,firstName,middleName,suffix,entityName,city,state,mailZipCode,licensePrefix,subCategory,...,licenseStatusDescription,specialty,title,degrees,caseNumber,programAction,disciplineEffectiveDate,disciplineCompleteDate,linkToVerifyLicense,linkToViewHealthcareProfile
0,Charles,James,Robert,NaN,NaN,Golden,CO,80401,APN,CRNA,...,Active,NaN,NaN,~Bachelors of Science in Nursing,NaN,NaN,NaN,NaN,https://www.colorado.gov/dora/licensing/Lookup...,NaN
1,Stagg,Russell,Adair,NaN,NaN,Meeker,CO,81641,C-APN,C-CRNA,...,Expired,NaN,NaN,~Assoc of Science in Nursing,NaN,NaN,NaN,NaN,https://www.colorado.gov/dora/licensing/Lookup...,https://www.colorado.gov/dora/licensing/Snapsh...
2,Sampier,Nicole,Louise A,NaN,NaN,Tampa,FL,33602,APN,CRNA,...,Active,NaN,NaN,~Bachelors of Science in Nursing,NaN,NaN,NaN,NaN,https://www.colorado.gov/dora/licensing/Lookup...,NaN
3,Trouskie,William,Leonard,JR,NaN,Rifle,CO,81650,APN,CRNA,...,Active,NaN,NaN,~Bachelors of Science in Nursing,NaN,NaN,NaN,NaN,https://www.colorado.gov/dora/licensing/Lookup...,NaN
4,Penix,Patrick,Alexander,NaN,NaN,Fort Collins,CO,80526,APN,CRNA,...,Active,NaN,NaN,~Bachelors of Science in Nursing,NaN,NaN,NaN,NaN,https://www.colorado.gov/dora/licensing/Lookup...,NaN


In [274]:
#anesthesia.info()

In [275]:
# Create a last plus first name matching element to merge dataframes
anesthesia['matchName'] = anesthesia['lastName'] + anesthesia['firstName']
anesthesia.head()

,lastName,firstName,middleName,suffix,entityName,city,state,mailZipCode,licensePrefix,subCategory,...,specialty,title,degrees,caseNumber,programAction,disciplineEffectiveDate,disciplineCompleteDate,linkToVerifyLicense,linkToViewHealthcareProfile,matchName
0,Charles,James,Robert,NaN,NaN,Golden,CO,80401,APN,CRNA,...,NaN,NaN,~Bachelors of Science in Nursing,NaN,NaN,NaN,NaN,https://www.colorado.gov/dora/licensing/Lookup...,NaN,CharlesJames
1,Stagg,Russell,Adair,NaN,NaN,Meeker,CO,81641,C-APN,C-CRNA,...,NaN,NaN,~Assoc of Science in Nursing,NaN,NaN,NaN,NaN,https://www.colorado.gov/dora/licensing/Lookup...,https://www.colorado.gov/dora/licensing/Snapsh...,StaggRussell
2,Sampier,Nicole,Louise A,NaN,NaN,Tampa,FL,33602,APN,CRNA,...,NaN,NaN,~Bachelors of Science in Nursing,NaN,NaN,NaN,NaN,https://www.colorado.gov/dora/licensing/Lookup...,NaN,SampierNicole
3,Trouskie,William,Leonard,JR,NaN,Rifle,CO,81650,APN,CRNA,...,NaN,NaN,~Bachelors of Science in Nursing,NaN,NaN,NaN,NaN,https://www.colorado.gov/dora/licensing/Lookup...,NaN,TrouskieWilliam
4,Penix,Patrick,Alexander,NaN,NaN,Fort Collins,CO,80526,APN,CRNA,...,NaN,NaN,~Bachelors of Science in Nursing,NaN,NaN,NaN,NaN,https://www.colorado.gov/dora/licensing/Lookup...,NaN,PenixPatrick


In [276]:
# Remove excess columns
del anesthesia["suffix"]
del anesthesia["entityName"]
del anesthesia["specialty"]
del anesthesia["title"]
del anesthesia["degrees"]
del anesthesia["linkToVerifyLicense"]
del anesthesia["linkToViewHealthcareProfile"]
anesthesia.head()

,lastName,firstName,middleName,city,state,mailZipCode,licensePrefix,subCategory,licenseNumber,licenseFirstIssueDate,licenseLastRenewedDate,licenseExpirationDate,licenseStatusDescription,caseNumber,programAction,disciplineEffectiveDate,disciplineCompleteDate,matchName
0,Charles,James,Robert,Golden,CO,80401,APN,CRNA,3656,02/15/2002 12:00:00 AM,10/01/2019 12:00:00 AM,09/30/2021 12:00:00 AM,Active,NaN,NaN,NaN,NaN,CharlesJames
1,Stagg,Russell,Adair,Meeker,CO,81641,C-APN,C-CRNA,211,06/18/2014 12:00:00 AM,06/18/2014 12:00:00 AM,11/05/2014 12:00:00 AM,Expired,NaN,NaN,NaN,NaN,StaggRussell
2,Sampier,Nicole,Louise A,Tampa,FL,33602,APN,CRNA,994392,01/03/2019 12:00:00 AM,10/01/2019 12:00:00 AM,09/30/2021 12:00:00 AM,Active,NaN,NaN,NaN,NaN,SampierNicole
3,Trouskie,William,Leonard,Rifle,CO,81650,APN,CRNA,15066,03/02/2010 12:00:00 AM,10/01/2020 12:00:00 AM,09/30/2022 12:00:00 AM,Active,NaN,NaN,NaN,NaN,TrouskieWilliam
4,Penix,Patrick,Alexander,Fort Collins,CO,80526,APN,CRNA,995315,01/27/2020 12:00:00 AM,10/01/2020 12:00:00 AM,09/30/2022 12:00:00 AM,Active,NaN,NaN,NaN,NaN,PenixPatrick


In [277]:
# Rename state to st because it looks like postgres is going to choke on it 
anesthesia = anesthesia.rename(columns={"state" : "st"})

In [278]:
anesthesia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2448 entries, 0 to 2447
Data columns (total 18 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   lastName                  2448 non-null   object
 1   firstName                 2448 non-null   object
 2   middleName                2200 non-null   object
 3   city                      2448 non-null   object
 4   st                        2448 non-null   object
 5   mailZipCode               2447 non-null   object
 6   licensePrefix             2448 non-null   object
 7   subCategory               2310 non-null   object
 8   licenseNumber             2448 non-null   int64 
 9   licenseFirstIssueDate     2448 non-null   object
 10  licenseLastRenewedDate    2447 non-null   object
 11  licenseExpirationDate     2448 non-null   object
 12  licenseStatusDescription  2448 non-null   object
 13  caseNumber                7 non-null      object
 14  programAction           

In [279]:
# Stage cleaned anesthesia file for database build
anesthesia.to_csv("loadAnesthesiaLicense.csv", index=False, header=True)

In [280]:
# Confirm that last name on its own will not support a matching process
# anesthesia['lastName'].value_counts()

In [281]:
# Confirm that last name first name combo on its own will support a matching process or build exception process
# anesthesia['matchName'].value_counts()

In [282]:
# Import the file with the registered nurse information required to validate underlying licenses
nurse=pd.read_csv('license_regnurse.csv')
nurse.head()

C:\Users\rodge\anaconda3\envs\PythonData\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (3,4,7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,lastName,firstName,middleName,suffix,entityName,city,state,mailZipCode,licensePrefix,subCategory,...,licenseStatusDescription,specialty,title,degrees,caseNumber,programAction,disciplineEffectiveDate,disciplineCompleteDate,linkToVerifyLicense,linkToViewHealthcareProfile
0,Saracino,Jodi,Kay,NaN,NaN,Arvada,CO,80004,RN,NaN,...,Active,NaN,NaN,~Bachelors of Science in Nursing,NaN,NaN,NaN,NaN,https://www.colorado.gov/dora/licensing/Lookup...,https://www.colorado.gov/dora/licensing/Snapsh...
1,Neal,Carmen,Darlene,NaN,NaN,Longview,TX,75608,RN,NaN,...,Expired,NaN,NaN,~Assoc of Science in Nursing,NaN,NaN,NaN,NaN,https://www.colorado.gov/dora/licensing/Lookup...,https://www.colorado.gov/dora/licensing/Snapsh...
2,Freeland,Judith,A.,NaN,NaN,BAILEY,CO,80421,RN,NaN,...,Expired,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.colorado.gov/dora/licensing/Lookup...,NaN
3,Healy,Julie,E,NaN,NaN,Colorado Springs,CO,80920,RN,NaN,...,Active,NaN,NaN,~Associate Degree in Nursing,NaN,NaN,NaN,NaN,https://www.colorado.gov/dora/licensing/Lookup...,https://www.colorado.gov/dora/licensing/Snapsh...
4,Englade,Keith,Steven,NaN,NaN,Aurora,CO,80013,RN,NaN,...,Active,NaN,NaN,~Bachelors of Science in Nursing,NaN,NaN,NaN,NaN,https://www.colorado.gov/dora/licensing/Lookup...,https://www.colorado.gov/dora/licensing/Snapsh...


In [283]:
# Create a last plus first name matching element to merge dataframes
nurse['matchName'] = nurse['lastName'] + nurse['firstName']
nurse.head()

,lastName,firstName,middleName,suffix,entityName,city,state,mailZipCode,licensePrefix,subCategory,...,specialty,title,degrees,caseNumber,programAction,disciplineEffectiveDate,disciplineCompleteDate,linkToVerifyLicense,linkToViewHealthcareProfile,matchName
0,Saracino,Jodi,Kay,NaN,NaN,Arvada,CO,80004,RN,NaN,...,NaN,NaN,~Bachelors of Science in Nursing,NaN,NaN,NaN,NaN,https://www.colorado.gov/dora/licensing/Lookup...,https://www.colorado.gov/dora/licensing/Snapsh...,SaracinoJodi
1,Neal,Carmen,Darlene,NaN,NaN,Longview,TX,75608,RN,NaN,...,NaN,NaN,~Assoc of Science in Nursing,NaN,NaN,NaN,NaN,https://www.colorado.gov/dora/licensing/Lookup...,https://www.colorado.gov/dora/licensing/Snapsh...,NealCarmen
2,Freeland,Judith,A.,NaN,NaN,BAILEY,CO,80421,RN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.colorado.gov/dora/licensing/Lookup...,NaN,FreelandJudith
3,Healy,Julie,E,NaN,NaN,Colorado Springs,CO,80920,RN,NaN,...,NaN,NaN,~Associate Degree in Nursing,NaN,NaN,NaN,NaN,https://www.colorado.gov/dora/licensing/Lookup...,https://www.colorado.gov/dora/licensing/Snapsh...,HealyJulie
4,Englade,Keith,Steven,NaN,NaN,Aurora,CO,80013,RN,NaN,...,NaN,NaN,~Bachelors of Science in Nursing,NaN,NaN,NaN,NaN,https://www.colorado.gov/dora/licensing/Lookup...,https://www.colorado.gov/dora/licensing/Snapsh...,EngladeKeith


In [284]:
#nurse.info()

In [285]:
# Remove unneeded columns
del nurse["suffix"]
del nurse["entityName"]
del nurse["subCategory"]
del nurse["specialty"]
del nurse["title"]
del nurse["licenseFirstIssueDate"]
del nurse["licenseLastRenewedDate"]
del nurse["degrees"]
del nurse["caseNumber"]
del nurse["programAction"]
del nurse["disciplineEffectiveDate"]
del nurse["disciplineCompleteDate"]
del nurse["linkToVerifyLicense"]
del nurse["linkToViewHealthcareProfile"]
nurse.head()

,lastName,firstName,middleName,city,state,mailZipCode,licensePrefix,licenseNumber,licenseExpirationDate,licenseStatusDescription,matchName
0,Saracino,Jodi,Kay,Arvada,CO,80004,RN,103136.0,09/30/2021 12:00:00 AM,Active,SaracinoJodi
1,Neal,Carmen,Darlene,Longview,TX,75608,RN,181576.0,09/30/2015 12:00:00 AM,Expired,NealCarmen
2,Freeland,Judith,A.,BAILEY,CO,80421,RN,92692.0,09/30/2005 12:00:00 AM,Expired,FreelandJudith
3,Healy,Julie,E,Colorado Springs,CO,80920,RN,130994.0,09/30/2021 12:00:00 AM,Active,HealyJulie
4,Englade,Keith,Steven,Aurora,CO,80013,RN,205425.0,09/30/2022 12:00:00 AM,Active,EngladeKeith


In [286]:
# Rename state to st because it looks like postgres is going to choke on it 
nurse = nurse.rename(columns={"state" : "st"})

In [287]:
nurse.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196995 entries, 0 to 196994
Data columns (total 11 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   lastName                  196994 non-null  object 
 1   firstName                 196993 non-null  object 
 2   middleName                180778 non-null  object 
 3   city                      196194 non-null  object 
 4   st                        194724 non-null  object 
 5   mailZipCode               195656 non-null  object 
 6   licensePrefix             196995 non-null  object 
 7   licenseNumber             196991 non-null  float64
 8   licenseExpirationDate     196994 non-null  object 
 9   licenseStatusDescription  196995 non-null  object 
 10  matchName                 196993 non-null  object 
dtypes: float64(1), object(10)
memory usage: 16.5+ MB


In [288]:
# Stage cleaned registered nurse license file for database build
nurse.to_csv("loadNurseLicense.csv", index=False, header=True)

In [289]:
#Import the employee roster (a random subset of public licensees used as fictitious roster in lieu of actual employee roster)
employee=pd.read_csv('employee_roster.csv')
employee.head()

,lastName,firstName,middleName,employeeNum,city,state,mailZipCode,title,hireDate,projectIndicator
0,Ackerman,Serena,Maija,97,Mount Shasta,CA,96067,CRNA,3/16/2009 0:00,1
1,Adams,Chad,R,1049,Lexington,KY,40514,CRNA,7/2/2018 0:00,1
2,Ahern,Caitlin,Moira,1260,Englewood,CO,80112,CRNA,10/23/2020 0:00,2
3,Balyer,Ean,NaN,859,Madison,WI,53703,AA,3/20/2019 0:00,1
4,Barfield,Benjamin,James,67,Denver,CO,80238,AA,6/5/2013 0:00,1


In [290]:
# Create a last plus first name matching element to merge dataframes
employee['matchName'] = employee['lastName'] + employee['firstName']
employee.head()

,lastName,firstName,middleName,employeeNum,city,state,mailZipCode,title,hireDate,projectIndicator,matchName
0,Ackerman,Serena,Maija,97,Mount Shasta,CA,96067,CRNA,3/16/2009 0:00,1,AckermanSerena
1,Adams,Chad,R,1049,Lexington,KY,40514,CRNA,7/2/2018 0:00,1,AdamsChad
2,Ahern,Caitlin,Moira,1260,Englewood,CO,80112,CRNA,10/23/2020 0:00,2,AhernCaitlin
3,Balyer,Ean,NaN,859,Madison,WI,53703,AA,3/20/2019 0:00,1,BalyerEan
4,Barfield,Benjamin,James,67,Denver,CO,80238,AA,6/5/2013 0:00,1,BarfieldBenjamin


In [291]:
# Remove excess columns
del employee["projectIndicator"]
del employee["city"]
del employee["state"]
del employee["mailZipCode"]
employee.head()

,lastName,firstName,middleName,employeeNum,title,hireDate,matchName
0,Ackerman,Serena,Maija,97,CRNA,3/16/2009 0:00,AckermanSerena
1,Adams,Chad,R,1049,CRNA,7/2/2018 0:00,AdamsChad
2,Ahern,Caitlin,Moira,1260,CRNA,10/23/2020 0:00,AhernCaitlin
3,Balyer,Ean,NaN,859,AA,3/20/2019 0:00,BalyerEan
4,Barfield,Benjamin,James,67,AA,6/5/2013 0:00,BarfieldBenjamin


In [292]:
employee.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79 entries, 0 to 78
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   lastName     79 non-null     object
 1   firstName    79 non-null     object
 2   middleName   67 non-null     object
 3   employeeNum  79 non-null     int64 
 4   title        79 non-null     object
 5   hireDate     79 non-null     object
 6   matchName    79 non-null     object
dtypes: int64(1), object(6)
memory usage: 4.4+ KB


In [293]:
# Stage cleaned employee roster file for database build
employee.to_csv("loadEmployeeRoster.csv", index=False, header=True)

In [294]:
# Compliance Reporting Section

# CSV to Pandas -  Anesthesia License Management Section

In [295]:
# One - Merge the employee and anesthesia license status dataframes using a left join
astatus = pd.merge(employee, anesthesia, on="matchName", how="left")
astatus.head()

,lastName_x,firstName_x,middleName_x,employeeNum,title,hireDate,matchName,lastName_y,firstName_y,middleName_y,...,subCategory,licenseNumber,licenseFirstIssueDate,licenseLastRenewedDate,licenseExpirationDate,licenseStatusDescription,caseNumber,programAction,disciplineEffectiveDate,disciplineCompleteDate
0,Ackerman,Serena,Maija,97,CRNA,3/16/2009 0:00,AckermanSerena,Ackerman,Serena,Maija,...,CRNA,6026,03/16/2009 12:00:00 AM,09/06/2019 12:00:00 AM,09/30/2021 12:00:00 AM,Active,NaN,NaN,NaN,NaN
1,Adams,Chad,R,1049,CRNA,7/2/2018 0:00,AdamsChad,Adams,Chad,R,...,C-CRNA,1154,07/02/2018 12:00:00 AM,10/01/2020 12:00:00 AM,09/30/2022 12:00:00 AM,Active,NaN,NaN,NaN,NaN
2,Ahern,Caitlin,Moira,1260,CRNA,10/23/2020 0:00,AhernCaitlin,Ahern,Caitlin,Moira,...,CRNA,995962,10/23/2020 12:00:00 AM,10/23/2020 12:00:00 AM,09/30/2021 12:00:00 AM,Active,NaN,NaN,NaN,NaN
3,Ahern,Caitlin,Moira,1260,CRNA,10/23/2020 0:00,AhernCaitlin,Ahern,Caitlin,Moira,...,C-CRNA,2009,04/08/2020 12:00:00 AM,10/01/2020 12:00:00 AM,10/23/2020 12:00:00 AM,Expired,NaN,NaN,NaN,NaN
4,Balyer,Ean,NaN,859,AA,3/20/2019 0:00,BalyerEan,Balyer,Ean,NaN,...,NaN,109,03/20/2019 12:00:00 AM,02/01/2020 12:00:00 AM,01/31/2022 12:00:00 AM,Active,NaN,NaN,NaN,NaN


In [296]:
astatus.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 101 entries, 0 to 100
Data columns (total 24 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   lastName_x                101 non-null    object
 1   firstName_x               101 non-null    object
 2   middleName_x              84 non-null     object
 3   employeeNum               101 non-null    int64 
 4   title                     101 non-null    object
 5   hireDate                  101 non-null    object
 6   matchName                 101 non-null    object
 7   lastName_y                101 non-null    object
 8   firstName_y               101 non-null    object
 9   middleName_y              84 non-null     object
 10  city                      101 non-null    object
 11  st                        101 non-null    object
 12  mailZipCode               101 non-null    object
 13  licensePrefix             101 non-null    object
 14  subCategory               

In [297]:
# TWo - Sort by matchName then licenseStatus, "Active" first
astatus.sort_values(by=['matchName', 'licenseStatusDescription'], inplace=True)
astatus.head()

,lastName_x,firstName_x,middleName_x,employeeNum,title,hireDate,matchName,lastName_y,firstName_y,middleName_y,...,subCategory,licenseNumber,licenseFirstIssueDate,licenseLastRenewedDate,licenseExpirationDate,licenseStatusDescription,caseNumber,programAction,disciplineEffectiveDate,disciplineCompleteDate
0,Ackerman,Serena,Maija,97,CRNA,3/16/2009 0:00,AckermanSerena,Ackerman,Serena,Maija,...,CRNA,6026,03/16/2009 12:00:00 AM,09/06/2019 12:00:00 AM,09/30/2021 12:00:00 AM,Active,NaN,NaN,NaN,NaN
1,Adams,Chad,R,1049,CRNA,7/2/2018 0:00,AdamsChad,Adams,Chad,R,...,C-CRNA,1154,07/02/2018 12:00:00 AM,10/01/2020 12:00:00 AM,09/30/2022 12:00:00 AM,Active,NaN,NaN,NaN,NaN
2,Ahern,Caitlin,Moira,1260,CRNA,10/23/2020 0:00,AhernCaitlin,Ahern,Caitlin,Moira,...,CRNA,995962,10/23/2020 12:00:00 AM,10/23/2020 12:00:00 AM,09/30/2021 12:00:00 AM,Active,NaN,NaN,NaN,NaN
3,Ahern,Caitlin,Moira,1260,CRNA,10/23/2020 0:00,AhernCaitlin,Ahern,Caitlin,Moira,...,C-CRNA,2009,04/08/2020 12:00:00 AM,10/01/2020 12:00:00 AM,10/23/2020 12:00:00 AM,Expired,NaN,NaN,NaN,NaN
4,Balyer,Ean,NaN,859,AA,3/20/2019 0:00,BalyerEan,Balyer,Ean,NaN,...,NaN,109,03/20/2019 12:00:00 AM,02/01/2020 12:00:00 AM,01/31/2022 12:00:00 AM,Active,NaN,NaN,NaN,NaN


In [298]:
# Three Remove duplicates leaving the active license in the compliance reporting process if it exists
astatus.drop_duplicates(subset ="matchName", keep = False, inplace = True) 
#astatus.to_csv("report_anesthesia_compliance.csv", index=False, header=True)
astatus.head()

,lastName_x,firstName_x,middleName_x,employeeNum,title,hireDate,matchName,lastName_y,firstName_y,middleName_y,...,subCategory,licenseNumber,licenseFirstIssueDate,licenseLastRenewedDate,licenseExpirationDate,licenseStatusDescription,caseNumber,programAction,disciplineEffectiveDate,disciplineCompleteDate
0,Ackerman,Serena,Maija,97,CRNA,3/16/2009 0:00,AckermanSerena,Ackerman,Serena,Maija,...,CRNA,6026,03/16/2009 12:00:00 AM,09/06/2019 12:00:00 AM,09/30/2021 12:00:00 AM,Active,NaN,NaN,NaN,NaN
1,Adams,Chad,R,1049,CRNA,7/2/2018 0:00,AdamsChad,Adams,Chad,R,...,C-CRNA,1154,07/02/2018 12:00:00 AM,10/01/2020 12:00:00 AM,09/30/2022 12:00:00 AM,Active,NaN,NaN,NaN,NaN
4,Balyer,Ean,NaN,859,AA,3/20/2019 0:00,BalyerEan,Balyer,Ean,NaN,...,NaN,109,03/20/2019 12:00:00 AM,02/01/2020 12:00:00 AM,01/31/2022 12:00:00 AM,Active,NaN,NaN,NaN,NaN
5,Barfield,Benjamin,James,67,AA,6/5/2013 0:00,BarfieldBenjamin,Barfield,Benjamin,James,...,NaN,10,06/05/2013 12:00:00 AM,02/01/2020 12:00:00 AM,01/31/2022 12:00:00 AM,Active,NaN,NaN,NaN,NaN
8,Boudreaux,David,Michael,551,CRNA,10/12/2010 0:00,BoudreauxDavid,Boudreaux,David,Michael,...,CRNA,100010,10/12/2010 12:00:00 AM,10/01/2020 12:00:00 AM,09/30/2022 12:00:00 AM,Active,NaN,NaN,NaN,NaN


In [299]:
astatus.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 60 entries, 0 to 100
Data columns (total 24 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   lastName_x                60 non-null     object
 1   firstName_x               60 non-null     object
 2   middleName_x              53 non-null     object
 3   employeeNum               60 non-null     int64 
 4   title                     60 non-null     object
 5   hireDate                  60 non-null     object
 6   matchName                 60 non-null     object
 7   lastName_y                60 non-null     object
 8   firstName_y               60 non-null     object
 9   middleName_y              53 non-null     object
 10  city                      60 non-null     object
 11  st                        60 non-null     object
 12  mailZipCode               60 non-null     object
 13  licensePrefix             60 non-null     object
 14  subCategory               5

In [300]:
# Four Produce a red flag report of out of compliance anesthesia clinicians
redflag = astatus[(astatus["licenseStatusDescription"] == "Expired")]
#redflag.to_csv("report_anesthesia_redflag.csv", index=False, header=True)
redflag.head()

,lastName_x,firstName_x,middleName_x,employeeNum,title,hireDate,matchName,lastName_y,firstName_y,middleName_y,...,subCategory,licenseNumber,licenseFirstIssueDate,licenseLastRenewedDate,licenseExpirationDate,licenseStatusDescription,caseNumber,programAction,disciplineEffectiveDate,disciplineCompleteDate
81,Smith,Catherine,Vu,1255,CRNA,11/24/2020 0:00,SmithCatherine,Smith,Catherine,Vu,...,C-CRNA,1082,05/21/2018 12:00:00 AM,10/01/2020 12:00:00 AM,11/24/2020 12:00:00 AM,Expired,NaN,NaN,NaN,NaN


# CSV to Pandas -  Registered Nurse Sub-License Management Section

In [213]:
# One - Merge the employee and registered nurse license status dataframes using a left join
rn_status = pd.merge(employee, nurse, on="matchName", how="left")
rn_status.head()

,lastName_x,firstName_x,middleName_x,employeeNum,title,hireDate,matchName,lastName_y,firstName_y,middleName_y,city,st,mailZipCode,licensePrefix,licenseNumber,licenseExpirationDate,licenseStatusDescription
0,Ackerman,Serena,Maija,97,CRNA,3/16/2009 0:00,AckermanSerena,Ackerman,Serena,Maija,Mount Shasta,CA,96067,RN,181740.0,09/30/2021 12:00:00 AM,Active
1,Adams,Chad,R,1049,CRNA,7/2/2018 0:00,AdamsChad,Adams,Chad,A,Longmont,CO,80503,RN,167534.0,09/30/2021 12:00:00 AM,Active
2,Ahern,Caitlin,Moira,1260,CRNA,10/23/2020 0:00,AhernCaitlin,Ahern,Caitlin,Moira,Englewood,CO,80112,RN,1622686.0,09/30/2021 12:00:00 AM,Active
3,Balyer,Ean,NaN,859,AA,3/20/2019 0:00,BalyerEan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Barfield,Benjamin,James,67,AA,6/5/2013 0:00,BarfieldBenjamin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [214]:
# Two - Sort by matchName then licenseStatus, "Active" first
rn_status.sort_values(by=['matchName', 'licenseStatusDescription'], inplace=True)
rn_status.head()

,lastName_x,firstName_x,middleName_x,employeeNum,title,hireDate,matchName,lastName_y,firstName_y,middleName_y,city,st,mailZipCode,licensePrefix,licenseNumber,licenseExpirationDate,licenseStatusDescription
0,Ackerman,Serena,Maija,97,CRNA,3/16/2009 0:00,AckermanSerena,Ackerman,Serena,Maija,Mount Shasta,CA,96067,RN,181740.0,09/30/2021 12:00:00 AM,Active
1,Adams,Chad,R,1049,CRNA,7/2/2018 0:00,AdamsChad,Adams,Chad,A,Longmont,CO,80503,RN,167534.0,09/30/2021 12:00:00 AM,Active
2,Ahern,Caitlin,Moira,1260,CRNA,10/23/2020 0:00,AhernCaitlin,Ahern,Caitlin,Moira,Englewood,CO,80112,RN,1622686.0,09/30/2021 12:00:00 AM,Active
3,Balyer,Ean,NaN,859,AA,3/20/2019 0:00,BalyerEan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Barfield,Benjamin,James,67,AA,6/5/2013 0:00,BarfieldBenjamin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [215]:
# Three - Remove duplicates leaving the "Active" status if true
rn_status.drop_duplicates(subset ="matchName", keep = False, inplace = True) 
rn_status.head()

,lastName_x,firstName_x,middleName_x,employeeNum,title,hireDate,matchName,lastName_y,firstName_y,middleName_y,city,st,mailZipCode,licensePrefix,licenseNumber,licenseExpirationDate,licenseStatusDescription
0,Ackerman,Serena,Maija,97,CRNA,3/16/2009 0:00,AckermanSerena,Ackerman,Serena,Maija,Mount Shasta,CA,96067,RN,181740.0,09/30/2021 12:00:00 AM,Active
1,Adams,Chad,R,1049,CRNA,7/2/2018 0:00,AdamsChad,Adams,Chad,A,Longmont,CO,80503,RN,167534.0,09/30/2021 12:00:00 AM,Active
2,Ahern,Caitlin,Moira,1260,CRNA,10/23/2020 0:00,AhernCaitlin,Ahern,Caitlin,Moira,Englewood,CO,80112,RN,1622686.0,09/30/2021 12:00:00 AM,Active
3,Balyer,Ean,NaN,859,AA,3/20/2019 0:00,BalyerEan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Barfield,Benjamin,James,67,AA,6/5/2013 0:00,BarfieldBenjamin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [216]:
# Four - Remove Anesthesiologist Assistants because they do not require an RN sublicense and produce a compliance report
rn_status = rn_status[(rn_status["title"] == "CRNA")]
#rn_status.to_csv("report_nurse_compliance.csv", index=False, header=True)
rn_status.head()

,lastName_x,firstName_x,middleName_x,employeeNum,title,hireDate,matchName,lastName_y,firstName_y,middleName_y,city,st,mailZipCode,licensePrefix,licenseNumber,licenseExpirationDate,licenseStatusDescription
0,Ackerman,Serena,Maija,97,CRNA,3/16/2009 0:00,AckermanSerena,Ackerman,Serena,Maija,Mount Shasta,CA,96067,RN,181740.0,09/30/2021 12:00:00 AM,Active
1,Adams,Chad,R,1049,CRNA,7/2/2018 0:00,AdamsChad,Adams,Chad,A,Longmont,CO,80503,RN,167534.0,09/30/2021 12:00:00 AM,Active
2,Ahern,Caitlin,Moira,1260,CRNA,10/23/2020 0:00,AhernCaitlin,Ahern,Caitlin,Moira,Englewood,CO,80112,RN,1622686.0,09/30/2021 12:00:00 AM,Active
5,Bishop,Trent,M,1258,CRNA,11/13/2020 0:00,BishopTrent,Bishop,Trent,M,Durango,CO,81301,RN,1671331.0,09/30/2022 12:00:00 AM,Active
6,Boudreaux,David,Michael,551,CRNA,10/12/2010 0:00,BoudreauxDavid,Boudreaux,David,Michael,New Orleans,LA,70119,RN,197160.0,09/30/2022 12:00:00 AM,Active


In [217]:
# Five - Produce a red flag list of anesthesia clinicians that do not have a required sublicense with "Active" status
rn_redflag = rn_status[(rn_status["licenseStatusDescription"] != "Active")]
#rn_redflag.to_csv("report_nurse_redflag.csv", index=False, header=True)
rn_redflag.head()

,lastName_x,firstName_x,middleName_x,employeeNum,title,hireDate,matchName,lastName_y,firstName_y,middleName_y,city,st,mailZipCode,licensePrefix,licenseNumber,licenseExpirationDate,licenseStatusDescription
20,Gates,Niki,Lynn,522,CRNA,6/19/2018 0:00,GatesNiki,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25,Jernigan,Amanda,Rhea,129,CRNA,3/14/2017 0:00,JerniganAmanda,Jernigan,Amanda,Rhea,Nashville,TN,37205,RN,195510.0,03/14/2017 12:00:00 AM,Expired
43,Lagrimas,Carl,NaN,775,CRNA,9/27/2019 0:00,LagrimasCarl,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44,Lam,Bao,Thai,370,CRNA,3/2/2018 0:00,LamBao,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
77,Sims,John,NaN,112,CRNA,12/13/2016 0:00,SimsJohn,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Quality and issue checks ...

In [218]:
# wieser = anesthesia[(anesthesia["lastName"] == "Wieser")]
# wieser.head()

In [219]:
# wieser = employee[(employee["lastName"] == "Wieser")]
# wieser.head()

In [220]:
# wieser = nurse[(nurse["lastName"] == "Wieser")]
# wieser.head(100)

In [221]:
# gates = anesthesia[(anesthesia["lastName"] == "Gates")]
# gates.head()

In [222]:
# gates = employee[(employee["lastName"] == "Gates")]
# gates.head()

In [223]:
# gatesplus = nurse[(nurse["lastName"] == "Gates") & (nurse["firstName"] == "Nikki")]
# gatesplus.head(100)

In [224]:
# other = anesthesia[(anesthesia["lastName"] == "Fishell")]
# other.head()

# Extracting from new PGAdmin Database

In [301]:
# Creates a connection to the DB
engine = create_engine('postgresql://postgres:'+ passcode + '@localhost:5432/aEtlChallenge')
conn = engine.connect()

In [302]:
# Preview the anesthesia license data
al = pd.read_sql("SELECT * FROM anesthesialicense", conn)
al.head()

,lastname,firstname,middlename,city,st,mailzipcode,licenseprefix,subcategory,licensenumber,licensefirstissuedate,licenselastreneweddate,licenseexpirationdate,licensestatusdescription,casenumber,programaction,disciplineeffectivedate,disciplinecompletedate,matchname
0,Charles,James,Robert,Golden,CO,80401,APN,CRNA,3656,02/15/2002 12:00:00 AM,10/01/2019 12:00:00 AM,09/30/2021 12:00:00 AM,Active,None,None,None,None,CharlesJames
1,Stagg,Russell,Adair,Meeker,CO,81641,C-APN,C-CRNA,211,06/18/2014 12:00:00 AM,06/18/2014 12:00:00 AM,11/05/2014 12:00:00 AM,Expired,None,None,None,None,StaggRussell
2,Sampier,Nicole,Louise A,Tampa,FL,33602,APN,CRNA,994392,01/03/2019 12:00:00 AM,10/01/2019 12:00:00 AM,09/30/2021 12:00:00 AM,Active,None,None,None,None,SampierNicole
3,Trouskie,William,Leonard,Rifle,CO,81650,APN,CRNA,15066,03/02/2010 12:00:00 AM,10/01/2020 12:00:00 AM,09/30/2022 12:00:00 AM,Active,None,None,None,None,TrouskieWilliam
4,Penix,Patrick,Alexander,Fort Collins,CO,80526,APN,CRNA,995315,01/27/2020 12:00:00 AM,10/01/2020 12:00:00 AM,09/30/2022 12:00:00 AM,Active,None,None,None,None,PenixPatrick


In [303]:
al.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2448 entries, 0 to 2447
Data columns (total 18 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   lastname                  2448 non-null   object
 1   firstname                 2448 non-null   object
 2   middlename                2200 non-null   object
 3   city                      2448 non-null   object
 4   st                        2448 non-null   object
 5   mailzipcode               2447 non-null   object
 6   licenseprefix             2448 non-null   object
 7   subcategory               2310 non-null   object
 8   licensenumber             2448 non-null   object
 9   licensefirstissuedate     2448 non-null   object
 10  licenselastreneweddate    2447 non-null   object
 11  licenseexpirationdate     2448 non-null   object
 12  licensestatusdescription  2448 non-null   object
 13  casenumber                7 non-null      object
 14  programaction           

In [304]:
# Preview the nurse license data
nl = pd.read_sql("SELECT * FROM nurselicense", conn)
nl.head()

,lastname,firstname,middlename,city,st,mailzipcode,licenseprefix,licensenumber,licenseexpirationdate,licensestatusdescription,matchname
0,Saracino,Jodi,Kay,Arvada,CO,80004,RN,103136.0,09/30/2021 12:00:00 AM,Active,SaracinoJodi
1,Neal,Carmen,Darlene,Longview,TX,75608,RN,181576.0,09/30/2015 12:00:00 AM,Expired,NealCarmen
2,Freeland,Judith,A.,BAILEY,CO,80421,RN,92692.0,09/30/2005 12:00:00 AM,Expired,FreelandJudith
3,Healy,Julie,E,Colorado Springs,CO,80920,RN,130994.0,09/30/2021 12:00:00 AM,Active,HealyJulie
4,Englade,Keith,Steven,Aurora,CO,80013,RN,205425.0,09/30/2022 12:00:00 AM,Active,EngladeKeith


In [305]:
nl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196995 entries, 0 to 196994
Data columns (total 11 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   lastname                  196994 non-null  object
 1   firstname                 196993 non-null  object
 2   middlename                180778 non-null  object
 3   city                      196194 non-null  object
 4   st                        194724 non-null  object
 5   mailzipcode               195656 non-null  object
 6   licenseprefix             196995 non-null  object
 7   licensenumber             196991 non-null  object
 8   licenseexpirationdate     196994 non-null  object
 9   licensestatusdescription  196995 non-null  object
 10  matchname                 196993 non-null  object
dtypes: object(11)
memory usage: 16.5+ MB


In [306]:
# Preview the employee_roster data
er = pd.read_sql("SELECT * FROM employeeroster", conn)
er.head()

,lastname,firstname,middlename,employeenum,title,hiredate,matchname
0,Ackerman,Serena,Maija,97,CRNA,3/16/2009 0:00,AckermanSerena
1,Adams,Chad,R,1049,CRNA,7/2/2018 0:00,AdamsChad
2,Ahern,Caitlin,Moira,1260,CRNA,10/23/2020 0:00,AhernCaitlin
3,Balyer,Ean,None,859,AA,3/20/2019 0:00,BalyerEan
4,Barfield,Benjamin,James,67,AA,6/5/2013 0:00,BarfieldBenjamin


In [307]:
er.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79 entries, 0 to 78
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   lastname     79 non-null     object
 1   firstname    79 non-null     object
 2   middlename   67 non-null     object
 3   employeenum  79 non-null     object
 4   title        79 non-null     object
 5   hiredate     79 non-null     object
 6   matchname    79 non-null     object
dtypes: object(7)
memory usage: 4.4+ KB


# PGAdmin to Pandas Anesthesia License Management

In [308]:
# One - Merge the employee and anesthesia license status dataframes using a left join
pgastatus = pd.merge(er, al, on="matchname", how="left")
pgastatus.head(200)

,lastname_x,firstname_x,middlename_x,employeenum,title,hiredate,matchname,lastname_y,firstname_y,middlename_y,...,subcategory,licensenumber,licensefirstissuedate,licenselastreneweddate,licenseexpirationdate,licensestatusdescription,casenumber,programaction,disciplineeffectivedate,disciplinecompletedate
0,Ackerman,Serena,Maija,97,CRNA,3/16/2009 0:00,AckermanSerena,Ackerman,Serena,Maija,...,CRNA,6026,03/16/2009 12:00:00 AM,09/06/2019 12:00:00 AM,09/30/2021 12:00:00 AM,Active,None,None,None,None
1,Adams,Chad,R,1049,CRNA,7/2/2018 0:00,AdamsChad,Adams,Chad,R,...,C-CRNA,1154,07/02/2018 12:00:00 AM,10/01/2020 12:00:00 AM,09/30/2022 12:00:00 AM,Active,None,None,None,None
2,Ahern,Caitlin,Moira,1260,CRNA,10/23/2020 0:00,AhernCaitlin,Ahern,Caitlin,Moira,...,CRNA,995962,10/23/2020 12:00:00 AM,10/23/2020 12:00:00 AM,09/30/2021 12:00:00 AM,Active,None,None,None,None
3,Ahern,Caitlin,Moira,1260,CRNA,10/23/2020 0:00,AhernCaitlin,Ahern,Caitlin,Moira,...,C-CRNA,2009,04/08/2020 12:00:00 AM,10/01/2020 12:00:00 AM,10/23/2020 12:00:00 AM,Expired,None,None,None,None
4,Balyer,Ean,None,859,AA,3/20/2019 0:00,BalyerEan,Balyer,Ean,None,...,None,109,03/20/2019 12:00:00 AM,02/01/2020 12:00:00 AM,01/31/2022 12:00:00 AM,Active,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,Wieser,Christopher,Lee,1215,CRNA,5/5/2020 0:00,WieserChristopher,Wieser,Christopher,Lee,...,C-CRNA,2056,05/05/2020 12:00:00 AM,10/01/2020 12:00:00 AM,09/30/2022 12:00:00 AM,Active,None,None,None,None
97,Woody,Patricia,McGuire,434,CRNA,1/23/2018 0:00,WoodyPatricia,Woody,Patricia,McGuire,...,CRNA,993651,01/23/2018 12:00:00 AM,10/01/2019 12:00:00 AM,09/30/2021 12:00:00 AM,Active,None,None,None,None
98,Woody,Patricia,McGuire,434,CRNA,1/23/2018 0:00,WoodyPatricia,Woody,Patricia,McGuire,...,C-CRNA,784,06/05/2017 12:00:00 AM,06/05/2017 12:00:00 AM,01/23/2018 12:00:00 AM,Expired,None,None,None,None
99,Wright,Henry,Wilson,824,CRNA,1/9/2007 0:00,WrightHenry,Wright,Henry,Wilson,...,CRNA,5158,01/09/2007 12:00:00 AM,10/01/2020 12:00:00 AM,09/30/2022 12:00:00 AM,Active,None,None,None,None


In [311]:
pgastatus['licenseexpirationdate'] = pd.to_datetime(pgastatus['licenseexpirationdate'], format = '%m%d%Y')

ValueError: time data '09/30/2021 12:00:00 AM' does not match format '%m%d%Y' (match)

In [262]:
pgastatus.to_csv("check_pgastatus.csv", index=False, header=True)

In [263]:
pgastatus.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 101 entries, 0 to 100
Data columns (total 24 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   lastname_x                101 non-null    object
 1   firstname_x               101 non-null    object
 2   middlename_x              84 non-null     object
 3   employeenum               101 non-null    object
 4   title                     101 non-null    object
 5   hiredate                  101 non-null    object
 6   matchname                 101 non-null    object
 7   lastname_y                101 non-null    object
 8   firstname_y               101 non-null    object
 9   middlename_y              84 non-null     object
 10  city                      101 non-null    object
 11  st                        101 non-null    object
 12  mailzipcode               101 non-null    object
 13  licenseprefix             101 non-null    object
 14  subcategory               

In [264]:
# TWo - Sort by matchName then licenseStatus, "Active" first
pgastatus.sort_values(by=['matchname', 'licensestatusdescription'], inplace=True)
pgastatus.head(200)

,lastname_x,firstname_x,middlename_x,employeenum,title,hiredate,matchname,lastname_y,firstname_y,middlename_y,...,subcategory,licensenumber,licensefirstissuedate,licenselastreneweddate,licenseexpirationdate,licensestatusdescription,casenumber,programaction,disciplineeffectivedate,disciplinecompletedate
0,Ackerman,Serena,Maija,97,CRNA,3/16/2009 0:00,AckermanSerena,Ackerman,Serena,Maija,...,CRNA,6026,03/16/2009 12:00:00 AM,09/06/2019 12:00:00 AM,09/30/2021 12:00:00 AM,Active,None,None,None,None
1,Adams,Chad,R,1049,CRNA,7/2/2018 0:00,AdamsChad,Adams,Chad,R,...,C-CRNA,1154,07/02/2018 12:00:00 AM,10/01/2020 12:00:00 AM,09/30/2022 12:00:00 AM,Active,None,None,None,None
2,Ahern,Caitlin,Moira,1260,CRNA,10/23/2020 0:00,AhernCaitlin,Ahern,Caitlin,Moira,...,CRNA,995962,10/23/2020 12:00:00 AM,10/23/2020 12:00:00 AM,09/30/2021 12:00:00 AM,Active,None,None,None,None
3,Ahern,Caitlin,Moira,1260,CRNA,10/23/2020 0:00,AhernCaitlin,Ahern,Caitlin,Moira,...,C-CRNA,2009,04/08/2020 12:00:00 AM,10/01/2020 12:00:00 AM,10/23/2020 12:00:00 AM,Expired,None,None,None,None
4,Balyer,Ean,None,859,AA,3/20/2019 0:00,BalyerEan,Balyer,Ean,None,...,None,109,03/20/2019 12:00:00 AM,02/01/2020 12:00:00 AM,01/31/2022 12:00:00 AM,Active,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,Wieser,Christopher,Lee,1215,CRNA,5/5/2020 0:00,WieserChristopher,Wieser,Christopher,Lee,...,C-CRNA,2056,05/05/2020 12:00:00 AM,10/01/2020 12:00:00 AM,09/30/2022 12:00:00 AM,Active,None,None,None,None
97,Woody,Patricia,McGuire,434,CRNA,1/23/2018 0:00,WoodyPatricia,Woody,Patricia,McGuire,...,CRNA,993651,01/23/2018 12:00:00 AM,10/01/2019 12:00:00 AM,09/30/2021 12:00:00 AM,Active,None,None,None,None
98,Woody,Patricia,McGuire,434,CRNA,1/23/2018 0:00,WoodyPatricia,Woody,Patricia,McGuire,...,C-CRNA,784,06/05/2017 12:00:00 AM,06/05/2017 12:00:00 AM,01/23/2018 12:00:00 AM,Expired,None,None,None,None
99,Wright,Henry,Wilson,824,CRNA,1/9/2007 0:00,WrightHenry,Wright,Henry,Wilson,...,CRNA,5158,01/09/2007 12:00:00 AM,10/01/2020 12:00:00 AM,09/30/2022 12:00:00 AM,Active,None,None,None,None


In [265]:
pgastatus.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 101 entries, 0 to 100
Data columns (total 24 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   lastname_x                101 non-null    object
 1   firstname_x               101 non-null    object
 2   middlename_x              84 non-null     object
 3   employeenum               101 non-null    object
 4   title                     101 non-null    object
 5   hiredate                  101 non-null    object
 6   matchname                 101 non-null    object
 7   lastname_y                101 non-null    object
 8   firstname_y               101 non-null    object
 9   middlename_y              84 non-null     object
 10  city                      101 non-null    object
 11  st                        101 non-null    object
 12  mailzipcode               101 non-null    object
 13  licenseprefix             101 non-null    object
 14  subcategory               

In [266]:
pgastatus.head()

,lastname_x,firstname_x,middlename_x,employeenum,title,hiredate,matchname,lastname_y,firstname_y,middlename_y,...,subcategory,licensenumber,licensefirstissuedate,licenselastreneweddate,licenseexpirationdate,licensestatusdescription,casenumber,programaction,disciplineeffectivedate,disciplinecompletedate
0,Ackerman,Serena,Maija,97,CRNA,3/16/2009 0:00,AckermanSerena,Ackerman,Serena,Maija,...,CRNA,6026,03/16/2009 12:00:00 AM,09/06/2019 12:00:00 AM,09/30/2021 12:00:00 AM,Active,None,None,None,None
1,Adams,Chad,R,1049,CRNA,7/2/2018 0:00,AdamsChad,Adams,Chad,R,...,C-CRNA,1154,07/02/2018 12:00:00 AM,10/01/2020 12:00:00 AM,09/30/2022 12:00:00 AM,Active,None,None,None,None
2,Ahern,Caitlin,Moira,1260,CRNA,10/23/2020 0:00,AhernCaitlin,Ahern,Caitlin,Moira,...,CRNA,995962,10/23/2020 12:00:00 AM,10/23/2020 12:00:00 AM,09/30/2021 12:00:00 AM,Active,None,None,None,None
3,Ahern,Caitlin,Moira,1260,CRNA,10/23/2020 0:00,AhernCaitlin,Ahern,Caitlin,Moira,...,C-CRNA,2009,04/08/2020 12:00:00 AM,10/01/2020 12:00:00 AM,10/23/2020 12:00:00 AM,Expired,None,None,None,None
4,Balyer,Ean,None,859,AA,3/20/2019 0:00,BalyerEan,Balyer,Ean,None,...,None,109,03/20/2019 12:00:00 AM,02/01/2020 12:00:00 AM,01/31/2022 12:00:00 AM,Active,None,None,None,None


In [267]:
# Three Remove duplicates leaving the active license in the compliance reporting process if it exists
# Produce a compliance report that has status of all employees
pgastatus.drop_duplicates(subset ="matchname", keep = False, inplace = True) 
#pgastatus.to_csv("report_anesthesia_compliance.csv", index=False, header=True)
pgastatus.head(200)

,lastname_x,firstname_x,middlename_x,employeenum,title,hiredate,matchname,lastname_y,firstname_y,middlename_y,...,subcategory,licensenumber,licensefirstissuedate,licenselastreneweddate,licenseexpirationdate,licensestatusdescription,casenumber,programaction,disciplineeffectivedate,disciplinecompletedate
0,Ackerman,Serena,Maija,97,CRNA,3/16/2009 0:00,AckermanSerena,Ackerman,Serena,Maija,...,CRNA,6026,03/16/2009 12:00:00 AM,09/06/2019 12:00:00 AM,09/30/2021 12:00:00 AM,Active,None,None,None,None
1,Adams,Chad,R,1049,CRNA,7/2/2018 0:00,AdamsChad,Adams,Chad,R,...,C-CRNA,1154,07/02/2018 12:00:00 AM,10/01/2020 12:00:00 AM,09/30/2022 12:00:00 AM,Active,None,None,None,None
4,Balyer,Ean,None,859,AA,3/20/2019 0:00,BalyerEan,Balyer,Ean,None,...,None,109,03/20/2019 12:00:00 AM,02/01/2020 12:00:00 AM,01/31/2022 12:00:00 AM,Active,None,None,None,None
5,Barfield,Benjamin,James,67,AA,6/5/2013 0:00,BarfieldBenjamin,Barfield,Benjamin,James,...,None,10,06/05/2013 12:00:00 AM,02/01/2020 12:00:00 AM,01/31/2022 12:00:00 AM,Active,None,None,None,None
8,Boudreaux,David,Michael,551,CRNA,10/12/2010 0:00,BoudreauxDavid,Boudreaux,David,Michael,...,CRNA,100010,10/12/2010 12:00:00 AM,10/01/2020 12:00:00 AM,09/30/2022 12:00:00 AM,Active,None,None,None,None
9,Bramm,Jonathan,David,926,AA,10/21/2013 0:00,BrammJonathan,Bramm,Jonathan,David,...,None,24,10/21/2013 12:00:00 AM,02/01/2020 12:00:00 AM,01/31/2022 12:00:00 AM,Active,None,None,None,None
10,Brunin,Tara,Suzanne,168,CRNA,8/9/2011 0:00,BruninTara,Brunin,Tara,Suzanne,...,CRNA,100049,08/09/2011 12:00:00 AM,10/01/2019 12:00:00 AM,09/30/2021 12:00:00 AM,Active,None,None,None,None
13,Carpenter,Leah,Sue,513,CRNA,3/30/2007 0:00,CarpenterLeah,Carpenter,Leah,Sue,...,CRNA,5252,03/30/2007 12:00:00 AM,10/01/2019 12:00:00 AM,09/30/2021 12:00:00 AM,Active,None,None,None,None
14,Christie,Timothy,Robert,139,CRNA,12/16/2010 0:00,ChristieTimothy,Christie,Timothy,Robert,...,CRNA,100021,12/16/2010 12:00:00 AM,10/01/2020 12:00:00 AM,09/30/2022 12:00:00 AM,Active,None,None,None,None
17,Croshal,Andrea,Lynn,906,AA,5/27/2016 0:00,CroshalAndrea,Croshal,Andrea,Lynn,...,None,61,05/27/2016 12:00:00 AM,02/01/2020 12:00:00 AM,01/31/2022 12:00:00 AM,Active,None,None,None,None


In [236]:
# Three-B rename compliance report
pgastatus2 = pgastatus.rename(columns={"lastname_x": "Last_Name", "firstname_x": "First_Name","middlename_x": "Middle_Name" ,"employeenum": "Employee_Number","title": "Title","hiredate": "Hire_Date","licensenumber": "License_Number","licenseexpirationdate": "Expiration_Date","licensestatusdescription": "License_Status"})
pgastatus2.head()

,Last_Name,First_Name,Middle_Name,Employee_Number,Title,Hire_Date,matchname,lastname_y,firstname_y,middlename_y,...,subcategory,License_Number,licensefirstissuedate,licenselastreneweddate,Expiration_Date,License_Status,casenumber,programaction,disciplineeffectivedate,disciplinecompletedate
0,Ackerman,Serena,Maija,97,CRNA,3/16/2009 0:00,AckermanSerena,Ackerman,Serena,Maija,...,CRNA,6026,03/16/2009 12:00:00 AM,09/06/2019 12:00:00 AM,09/30/2021 12:00:00 AM,Active,None,None,None,None
1,Adams,Chad,R,1049,CRNA,7/2/2018 0:00,AdamsChad,Adams,Chad,R,...,C-CRNA,1154,07/02/2018 12:00:00 AM,10/01/2020 12:00:00 AM,09/30/2022 12:00:00 AM,Active,None,None,None,None
4,Balyer,Ean,None,859,AA,3/20/2019 0:00,BalyerEan,Balyer,Ean,None,...,None,109,03/20/2019 12:00:00 AM,02/01/2020 12:00:00 AM,01/31/2022 12:00:00 AM,Active,None,None,None,None
5,Barfield,Benjamin,James,67,AA,6/5/2013 0:00,BarfieldBenjamin,Barfield,Benjamin,James,...,None,10,06/05/2013 12:00:00 AM,02/01/2020 12:00:00 AM,01/31/2022 12:00:00 AM,Active,None,None,None,None
8,Boudreaux,David,Michael,551,CRNA,10/12/2010 0:00,BoudreauxDavid,Boudreaux,David,Michael,...,CRNA,100010,10/12/2010 12:00:00 AM,10/01/2020 12:00:00 AM,09/30/2022 12:00:00 AM,Active,None,None,None,None


In [237]:
# Three-C Narrow the report to the needed information for consumption
pgastatus3 = pgastatus2[["Last_Name", "First_Name", "Middle_Name", "Employee_Number", "Title", "Hire_Date", "License_Number", "Expiration_Date", "License_Status"]]
pgastatus3.head()

,Last_Name,First_Name,Middle_Name,Employee_Number,Title,Hire_Date,License_Number,Expiration_Date,License_Status
0,Ackerman,Serena,Maija,97,CRNA,3/16/2009 0:00,6026,09/30/2021 12:00:00 AM,Active
1,Adams,Chad,R,1049,CRNA,7/2/2018 0:00,1154,09/30/2022 12:00:00 AM,Active
4,Balyer,Ean,None,859,AA,3/20/2019 0:00,109,01/31/2022 12:00:00 AM,Active
5,Barfield,Benjamin,James,67,AA,6/5/2013 0:00,10,01/31/2022 12:00:00 AM,Active
8,Boudreaux,David,Michael,551,CRNA,10/12/2010 0:00,100010,09/30/2022 12:00:00 AM,Active


In [238]:
# Three-D - Save to csv file for report consumption
pgastatus3.to_csv("report_anesthesia_compliance.csv", index=False, header=True)

In [239]:
# Four-A Produce a red flag report of out of compliance anesthesia clinicians
pgredflag = pgastatus[(pgastatus["licensestatusdescription"] == "Expired")]
pgredflag.head()

,lastname_x,firstname_x,middlename_x,employeenum,title,hiredate,matchname,lastname_y,firstname_y,middlename_y,...,subcategory,licensenumber,licensefirstissuedate,licenselastreneweddate,licenseexpirationdate,licensestatusdescription,casenumber,programaction,disciplineeffectivedate,disciplinecompletedate
81,Smith,Catherine,Vu,1255,CRNA,11/24/2020 0:00,SmithCatherine,Smith,Catherine,Vu,...,C-CRNA,1082,05/21/2018 12:00:00 AM,10/01/2020 12:00:00 AM,11/24/2020 12:00:00 AM,Expired,None,None,None,None


In [240]:
# Four-B rename redflag report
pgredflag2 = pgredflag.rename(columns={"lastname_x": "Last_Name", "firstname_x": "First_Name","middlename_x": "Middle_Name" ,"employeenum": "Employee_Number","title": "Title","hiredate": "Hire_Date","licensenumber": "License_Number","licenseexpirationdate": "Expiration_Date","licensestatusdescription": "License_Status"})
pgredflag2.head()

,Last_Name,First_Name,Middle_Name,Employee_Number,Title,Hire_Date,matchname,lastname_y,firstname_y,middlename_y,...,subcategory,License_Number,licensefirstissuedate,licenselastreneweddate,Expiration_Date,License_Status,casenumber,programaction,disciplineeffectivedate,disciplinecompletedate
81,Smith,Catherine,Vu,1255,CRNA,11/24/2020 0:00,SmithCatherine,Smith,Catherine,Vu,...,C-CRNA,1082,05/21/2018 12:00:00 AM,10/01/2020 12:00:00 AM,11/24/2020 12:00:00 AM,Expired,None,None,None,None


In [241]:
# Four-C Narrow the report to the needed information for consumption
pgredflag3 = pgredflag2[["Last_Name", "First_Name", "Middle_Name", "Employee_Number", "Title", "Hire_Date", "License_Number", "Expiration_Date", "License_Status"]]
pgredflag3.head()

,Last_Name,First_Name,Middle_Name,Employee_Number,Title,Hire_Date,License_Number,Expiration_Date,License_Status
81,Smith,Catherine,Vu,1255,CRNA,11/24/2020 0:00,1082,11/24/2020 12:00:00 AM,Expired


In [242]:
# Five - Save to csv file for report consumption
pgredflag3.to_csv("report_anesthesia_redflag.csv", index=False, header=True)

# PGAdmin to Pandas - Registered Nurse Sub-license Management

In [243]:
# One - Merge the employee and registered nurse license status table using a left join
pgrn_status = pd.merge(er, nl, on="matchname", how="left")
pgrn_status.head()

,lastname_x,firstname_x,middlename_x,employeenum,title,hiredate,matchname,lastname_y,firstname_y,middlename_y,city,st,mailzipcode,licenseprefix,licensenumber,licenseexpirationdate,licensestatusdescription
0,Ackerman,Serena,Maija,97,CRNA,3/16/2009 0:00,AckermanSerena,Ackerman,Serena,Maija,Mount Shasta,CA,96067,RN,181740.0,09/30/2021 12:00:00 AM,Active
1,Adams,Chad,R,1049,CRNA,7/2/2018 0:00,AdamsChad,Adams,Chad,A,Longmont,CO,80503,RN,167534.0,09/30/2021 12:00:00 AM,Active
2,Ahern,Caitlin,Moira,1260,CRNA,10/23/2020 0:00,AhernCaitlin,Ahern,Caitlin,Moira,Englewood,CO,80112,RN,1622686.0,09/30/2021 12:00:00 AM,Active
3,Balyer,Ean,None,859,AA,3/20/2019 0:00,BalyerEan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Barfield,Benjamin,James,67,AA,6/5/2013 0:00,BarfieldBenjamin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [244]:
# Two - Sort by matchName then licenseStatus, "Active" first
pgrn_status.sort_values(by=['matchname', 'licensestatusdescription'], inplace=True)
pgrn_status.head()

,lastname_x,firstname_x,middlename_x,employeenum,title,hiredate,matchname,lastname_y,firstname_y,middlename_y,city,st,mailzipcode,licenseprefix,licensenumber,licenseexpirationdate,licensestatusdescription
0,Ackerman,Serena,Maija,97,CRNA,3/16/2009 0:00,AckermanSerena,Ackerman,Serena,Maija,Mount Shasta,CA,96067,RN,181740.0,09/30/2021 12:00:00 AM,Active
1,Adams,Chad,R,1049,CRNA,7/2/2018 0:00,AdamsChad,Adams,Chad,A,Longmont,CO,80503,RN,167534.0,09/30/2021 12:00:00 AM,Active
2,Ahern,Caitlin,Moira,1260,CRNA,10/23/2020 0:00,AhernCaitlin,Ahern,Caitlin,Moira,Englewood,CO,80112,RN,1622686.0,09/30/2021 12:00:00 AM,Active
3,Balyer,Ean,None,859,AA,3/20/2019 0:00,BalyerEan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Barfield,Benjamin,James,67,AA,6/5/2013 0:00,BarfieldBenjamin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [245]:
# Three - Remove duplicates leaving the "Active" status if true
pgrn_status.drop_duplicates(subset ="matchname", keep = False, inplace = True) 
pgrn_status.head()

,lastname_x,firstname_x,middlename_x,employeenum,title,hiredate,matchname,lastname_y,firstname_y,middlename_y,city,st,mailzipcode,licenseprefix,licensenumber,licenseexpirationdate,licensestatusdescription
0,Ackerman,Serena,Maija,97,CRNA,3/16/2009 0:00,AckermanSerena,Ackerman,Serena,Maija,Mount Shasta,CA,96067,RN,181740.0,09/30/2021 12:00:00 AM,Active
1,Adams,Chad,R,1049,CRNA,7/2/2018 0:00,AdamsChad,Adams,Chad,A,Longmont,CO,80503,RN,167534.0,09/30/2021 12:00:00 AM,Active
2,Ahern,Caitlin,Moira,1260,CRNA,10/23/2020 0:00,AhernCaitlin,Ahern,Caitlin,Moira,Englewood,CO,80112,RN,1622686.0,09/30/2021 12:00:00 AM,Active
3,Balyer,Ean,None,859,AA,3/20/2019 0:00,BalyerEan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Barfield,Benjamin,James,67,AA,6/5/2013 0:00,BarfieldBenjamin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [246]:
# Four - Remove Anesthesiologist Assistants because they do not require an RN sublicense 
# Produce a compliance report with statsu of all CRNA's sub-license status
pgrn_status = pgrn_status[(pgrn_status["title"] == "CRNA")]
#pgrn_status.to_csv("report_rnsublicense_compliance.csv", index=False, header=True)
pgrn_status.head()

,lastname_x,firstname_x,middlename_x,employeenum,title,hiredate,matchname,lastname_y,firstname_y,middlename_y,city,st,mailzipcode,licenseprefix,licensenumber,licenseexpirationdate,licensestatusdescription
0,Ackerman,Serena,Maija,97,CRNA,3/16/2009 0:00,AckermanSerena,Ackerman,Serena,Maija,Mount Shasta,CA,96067,RN,181740.0,09/30/2021 12:00:00 AM,Active
1,Adams,Chad,R,1049,CRNA,7/2/2018 0:00,AdamsChad,Adams,Chad,A,Longmont,CO,80503,RN,167534.0,09/30/2021 12:00:00 AM,Active
2,Ahern,Caitlin,Moira,1260,CRNA,10/23/2020 0:00,AhernCaitlin,Ahern,Caitlin,Moira,Englewood,CO,80112,RN,1622686.0,09/30/2021 12:00:00 AM,Active
5,Bishop,Trent,M,1258,CRNA,11/13/2020 0:00,BishopTrent,Bishop,Trent,M,Durango,CO,81301,RN,1671331.0,09/30/2022 12:00:00 AM,Active
6,Boudreaux,David,Michael,551,CRNA,10/12/2010 0:00,BoudreauxDavid,Boudreaux,David,Michael,New Orleans,LA,70119,RN,197160.0,09/30/2022 12:00:00 AM,Active


In [247]:
# Four-B rename redflag report
pgrn_status2 = pgrn_status.rename(columns={"lastname_x": "Last_Name", "firstname_x": "First_Name","middlename_x": "Middle_Name" ,"employeenum": "Employee_Number","title": "Title","hiredate": "Hire_Date","licensenumber": "License_Number","licenseexpirationdate": "Expiration_Date","licensestatusdescription": "License_Status"})
pgrn_status2.head()

,Last_Name,First_Name,Middle_Name,Employee_Number,Title,Hire_Date,matchname,lastname_y,firstname_y,middlename_y,city,st,mailzipcode,licenseprefix,License_Number,Expiration_Date,License_Status
0,Ackerman,Serena,Maija,97,CRNA,3/16/2009 0:00,AckermanSerena,Ackerman,Serena,Maija,Mount Shasta,CA,96067,RN,181740.0,09/30/2021 12:00:00 AM,Active
1,Adams,Chad,R,1049,CRNA,7/2/2018 0:00,AdamsChad,Adams,Chad,A,Longmont,CO,80503,RN,167534.0,09/30/2021 12:00:00 AM,Active
2,Ahern,Caitlin,Moira,1260,CRNA,10/23/2020 0:00,AhernCaitlin,Ahern,Caitlin,Moira,Englewood,CO,80112,RN,1622686.0,09/30/2021 12:00:00 AM,Active
5,Bishop,Trent,M,1258,CRNA,11/13/2020 0:00,BishopTrent,Bishop,Trent,M,Durango,CO,81301,RN,1671331.0,09/30/2022 12:00:00 AM,Active
6,Boudreaux,David,Michael,551,CRNA,10/12/2010 0:00,BoudreauxDavid,Boudreaux,David,Michael,New Orleans,LA,70119,RN,197160.0,09/30/2022 12:00:00 AM,Active


In [248]:
# Four-C Narrow the report to the needed information for consumption
pgrn_status3 = pgrn_status2[["Last_Name", "First_Name", "Middle_Name", "Employee_Number", "Title", "Hire_Date", "License_Number", "Expiration_Date", "License_Status"]]
pgrn_status3.head()

,Last_Name,First_Name,Middle_Name,Employee_Number,Title,Hire_Date,License_Number,Expiration_Date,License_Status
0,Ackerman,Serena,Maija,97,CRNA,3/16/2009 0:00,181740.0,09/30/2021 12:00:00 AM,Active
1,Adams,Chad,R,1049,CRNA,7/2/2018 0:00,167534.0,09/30/2021 12:00:00 AM,Active
2,Ahern,Caitlin,Moira,1260,CRNA,10/23/2020 0:00,1622686.0,09/30/2021 12:00:00 AM,Active
5,Bishop,Trent,M,1258,CRNA,11/13/2020 0:00,1671331.0,09/30/2022 12:00:00 AM,Active
6,Boudreaux,David,Michael,551,CRNA,10/12/2010 0:00,197160.0,09/30/2022 12:00:00 AM,Active


In [251]:
# Four-D - Save to csv file for report consumption
pgrn_status3.to_csv("report_rnsublicense_compliance.csv", index=False, header=True)

In [252]:
# Five-A - Produce a red flag list of anesthesia clinicians that do not have a required sublicense with "Active" status 
pgrn_redflag = pgrn_status[(pgrn_status["licensestatusdescription"] != "Active")]
#pgrn_redflag.to_csv("report_pgnurse_redflag.csv", index=False, header=True)
pgrn_redflag.head()

,lastname_x,firstname_x,middlename_x,employeenum,title,hiredate,matchname,lastname_y,firstname_y,middlename_y,city,st,mailzipcode,licenseprefix,licensenumber,licenseexpirationdate,licensestatusdescription
20,Gates,Niki,Lynn,522,CRNA,6/19/2018 0:00,GatesNiki,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25,Jernigan,Amanda,Rhea,129,CRNA,3/14/2017 0:00,JerniganAmanda,Jernigan,Amanda,Rhea,Nashville,TN,37205,RN,195510.0,03/14/2017 12:00:00 AM,Expired
43,Lagrimas,Carl,None,775,CRNA,9/27/2019 0:00,LagrimasCarl,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44,Lam,Bao,Thai,370,CRNA,3/2/2018 0:00,LamBao,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
77,Sims,John,None,112,CRNA,12/13/2016 0:00,SimsJohn,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [253]:
# Five-B rename redflag report
pgrn_redflag2 = pgrn_redflag.rename(columns={"lastname_x": "Last_Name", "firstname_x": "First_Name","middlename_x": "Middle_Name" ,"employeenum": "Employee_Number","title": "Title","hiredate": "Hire_Date","licensenumber": "License_Number","licenseexpirationdate": "Expiration_Date","licensestatusdescription": "License_Status"})
pgrn_redflag2.head()

,Last_Name,First_Name,Middle_Name,Employee_Number,Title,Hire_Date,matchname,lastname_y,firstname_y,middlename_y,city,st,mailzipcode,licenseprefix,License_Number,Expiration_Date,License_Status
20,Gates,Niki,Lynn,522,CRNA,6/19/2018 0:00,GatesNiki,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25,Jernigan,Amanda,Rhea,129,CRNA,3/14/2017 0:00,JerniganAmanda,Jernigan,Amanda,Rhea,Nashville,TN,37205,RN,195510.0,03/14/2017 12:00:00 AM,Expired
43,Lagrimas,Carl,None,775,CRNA,9/27/2019 0:00,LagrimasCarl,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44,Lam,Bao,Thai,370,CRNA,3/2/2018 0:00,LamBao,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
77,Sims,John,None,112,CRNA,12/13/2016 0:00,SimsJohn,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [254]:
# Five-C Narrow the report to the needed information for consumption
pgrn_redflag3 = pgrn_redflag2[["Last_Name", "First_Name", "Middle_Name", "Employee_Number", "Title", "Hire_Date", "License_Number", "Expiration_Date", "License_Status"]]
pgrn_redflag3.head()

,Last_Name,First_Name,Middle_Name,Employee_Number,Title,Hire_Date,License_Number,Expiration_Date,License_Status
20,Gates,Niki,Lynn,522,CRNA,6/19/2018 0:00,NaN,NaN,NaN
25,Jernigan,Amanda,Rhea,129,CRNA,3/14/2017 0:00,195510.0,03/14/2017 12:00:00 AM,Expired
43,Lagrimas,Carl,None,775,CRNA,9/27/2019 0:00,NaN,NaN,NaN
44,Lam,Bao,Thai,370,CRNA,3/2/2018 0:00,NaN,NaN,NaN
77,Sims,John,None,112,CRNA,12/13/2016 0:00,NaN,NaN,NaN


In [255]:
# Six - Save to csv file for report consumption
pgrn_redflag3.to_csv("report_rnsublicense_redflag.csv", index=False, header=True)

# PGAdmin to Pandas - Anesthesia Cautionary Report

In [157]:
# Cautionary Report of License Due in next 90 days
#pgastatus90 = pgastatus(pgastatus[(licenseexpirationdate) < '2021-03-30']
#pgastatus90 = pgastatus[(pgastatus["licenseexpirationdate"] < "2021-03-30")]
pgastatus90 = pgastatus[(pgastatus["licenseexpirationdate"] < "3/30/2021")]
pgastatus90.to_csv("report_anesthesia_yellowflag.csv", index=False, header=True)
pgastatus90.head()

,lastname_x,firstname_x,middlename_x,employeenum,title,hiredate,matchname,lastname_y,firstname_y,middlename_y,...,subcategory,licensenumber,licensefirstissuedate,licenselastreneweddate,licenseexpirationdate,licensestatusdescription,casenumber,programaction,disciplineeffectivedate,disciplinecompletedate
0,Ackerman,Serena,Maija,97,CRNA,3/16/2009 0:00,AckermanSerena,Ackerman,Serena,Maija,...,CRNA,6026,03/16/2009 12:00:00 AM,09/06/2019 12:00:00 AM,09/30/2021 12:00:00 AM,Active,None,None,None,None
1,Adams,Chad,R,1049,CRNA,7/2/2018 0:00,AdamsChad,Adams,Chad,R,...,C-CRNA,1154,07/02/2018 12:00:00 AM,10/01/2020 12:00:00 AM,09/30/2022 12:00:00 AM,Active,None,None,None,None
4,Balyer,Ean,None,859,AA,3/20/2019 0:00,BalyerEan,Balyer,Ean,None,...,None,109,03/20/2019 12:00:00 AM,02/01/2020 12:00:00 AM,01/31/2022 12:00:00 AM,Active,None,None,None,None
5,Barfield,Benjamin,James,67,AA,6/5/2013 0:00,BarfieldBenjamin,Barfield,Benjamin,James,...,None,10,06/05/2013 12:00:00 AM,02/01/2020 12:00:00 AM,01/31/2022 12:00:00 AM,Active,None,None,None,None
8,Boudreaux,David,Michael,551,CRNA,10/12/2010 0:00,BoudreauxDavid,Boudreaux,David,Michael,...,CRNA,100010,10/12/2010 12:00:00 AM,10/01/2020 12:00:00 AM,09/30/2022 12:00:00 AM,Active,None,None,None,None


In [ ]:
# Rodgers ETL-Project End